In [14]:
import pandas as pd
import numpy as np
from IPython.display import display
from tqdm import tqdm

In [15]:
state_df = pd.read_csv('ahrs.csv')
control_df = pd.read_csv('vehicle_control.csv')
state_df['ts'] = pd.to_datetime(state_df['ts'])
control_df['ts'] = pd.to_datetime(control_df['ts'])

In [16]:
display(state_df.head())
display(control_df.head())

,ts,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps
0,2024-07-31 14:05:00.080599274+00:00,0.072542,-1.814206,340.940460,-9.894325,0.104737,-0.349738,-0.10250,-0.23250,-0.49500,-0.000328,0.000316,0.002213
1,2024-07-31 14:05:00.092972588+00:00,0.071586,-1.813645,340.940125,-9.969663,0.094937,-0.338713,-0.11750,-0.10375,-0.44750,-0.000357,0.000355,0.002445
2,2024-07-31 14:05:00.113763246+00:00,0.070876,-1.813810,340.938721,-9.963537,0.140262,-0.315438,-0.05125,-0.22125,-0.41750,-0.000010,0.000226,0.000702
3,2024-07-31 14:05:00.133055196+00:00,0.070295,-1.813962,340.939148,-9.992325,0.100450,-0.323400,-0.04250,-0.17250,-0.39125,-0.000253,0.000390,0.000410
4,2024-07-31 14:05:00.153150190+00:00,0.070045,-1.814164,340.939636,-10.025400,0.109637,-0.335650,-0.14375,-0.17375,-0.36750,-0.000029,0.000404,0.000288


,ts,gear,throttle,trim,turn
0,2024-07-31 14:05:00.082199113+00:00,0,0.008547,2,0.003663
1,2024-07-31 14:05:00.088896861+00:00,0,0.008547,2,0.003663
2,2024-07-31 14:05:00.094440420+00:00,0,0.008547,2,0.003663
3,2024-07-31 14:05:00.096188392+00:00,0,0.008547,2,0.003663
4,2024-07-31 14:05:00.105722298+00:00,0,0.008547,2,0.003663


In [17]:
state_df.describe()

,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps
count,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000,345001.000000
mean,-2.392771,2.790168,162.136319,-9.775801,-0.405489,0.449457,-0.259243,-0.165308,-0.339929,1.674247,0.930528,0.003550
std,3.058338,2.653283,117.193411,1.881210,1.013110,0.977098,2.750680,4.673068,5.118510,6.005039,5.177288,0.303745
min,-38.892082,-4.379096,0.000088,-75.060038,-75.563513,-22.327463,-49.990000,-32.725000,-57.185000,-11.874918,-11.601446,-3.617767
25%,-3.775711,1.248395,62.443455,-10.336550,-0.879550,-0.160475,-0.987500,-2.041250,-2.381250,-0.223773,-0.469180,-0.092668
50%,-2.333425,2.805565,126.043144,-9.793875,-0.287875,0.367500,-0.117500,-0.120000,-0.361250,0.125431,0.355929,0.000323
75%,-0.331669,4.888492,271.479889,-9.205262,0.064925,1.022875,0.678750,1.517500,1.623750,6.341228,5.052163,0.111911
max,23.512867,11.916366,359.999969,19.973625,27.339550,16.255750,27.857500,37.528750,59.587500,14.415796,13.385009,2.296954


In [18]:
control_df.describe()

,gear,throttle,trim,turn
count,537903.000000,537903.000000,537903.000000,537903.000000
mean,0.544014,0.223377,0.008087,0.003806
std,0.498059,0.217958,0.118245,0.145027
min,0.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.269841,0.000000,0.000000
75%,1.000000,0.450000,0.000000,0.003663
max,1.000000,0.688645,2.000000,1.000000


In [19]:
control_df['trim'].value_counts()

trim
0    535156
2      1603
1      1144
Name: count, dtype: int64

In [20]:
control_df['gear'].value_counts() # gear is never in reverse!

gear
1    292627
0    245276
Name: count, dtype: int64

In [21]:
def average_control_count_between_states(state_df, control_df, num_samples=100):
    # Ensure sorted
    state_df = state_df.sort_values('ts').reset_index(drop=True)
    control_df = control_df.sort_values('ts').reset_index(drop=True)

    control_counts = []
    count_windows = []

    max_idx = len(state_df) - 1
    sample_indices = np.random.choice(range(max_idx), size=num_samples, replace=False)

    for idx in tqdm(sample_indices):
        ts_t = state_df.loc[idx, 'ts']
        ts_tp1 = state_df.loc[idx + 1, 'ts']

        filtered = control_df.loc[(control_df['ts'] > ts_t) & (control_df['ts'] <= ts_tp1)]
        count = len(filtered)

        control_counts.append(count)
        count_windows.append((idx, filtered))  

    # Find the index of the max count
    max_count = max(control_counts)
    max_idx_in_sample = control_counts.index(max_count)
    max_state_idx, max_filtered_controls = count_windows[max_idx_in_sample]

    # Show the window
    print(f"Window with MAX number of controls ({max_count}): state index {max_state_idx}")
    display(state_df.iloc[max_state_idx:max_state_idx+1])
    display(state_df.iloc[max_state_idx+1:max_state_idx+2])
    display(max_filtered_controls)

    avg_count = np.mean(control_counts)
    min_count = min(control_counts)
    median_count = np.median(control_counts)
    std_count = np.std(control_counts)

    print(f"Average: {avg_count:.2f}, Max: {max_count}, Min: {min_count}, Median: {median_count}, Std: {std_count:.2f}")

    return avg_count, max_count, min_count, median_count, std_count

average_control_count_between_states(state_df, control_df)


100%|██████████| 100/100 [00:00<00:00, 1256.50it/s]

Window with MAX number of controls (6): state index 7072


,ts,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps
7072,2024-07-31 14:07:21.515189601+00:00,0.071567,-1.825128,340.93924,-9.717925,0.076562,-0.346675,-0.0775,-0.13375,-0.39,0.000495,0.000121,0.000859


,ts,roll_deg,pitch_deg,yaw_deg,ax_mps2,ay_mps2,az_mps2,omega_x_dps,omega_y_dps,omega_z_dps,ve_mps,vn_mps,vu_mps
7073,2024-07-31 14:07:21.639379625+00:00,0.071952,-1.82428,340.93985,-9.610125,0.065537,-0.3381,-0.0275,-0.14375,-0.3975,0.000702,0.000122,0.001322


,ts,gear,throttle,trim,turn
14753,2024-07-31 14:07:21.516583932+00:00,0,0.0,0,0.0
14754,2024-07-31 14:07:21.528957019+00:00,0,0.0,0,0.0
14755,2024-07-31 14:07:21.545781862+00:00,0,0.0,0,0.0
14756,2024-07-31 14:07:21.579413548+00:00,0,0.0,0,0.0
14757,2024-07-31 14:07:21.613043109+00:00,0,0.0,0,0.0
14758,2024-07-31 14:07:21.629881953+00:00,0,0.0,0,0.0


Average: 1.57, Max: 6, Min: 0, Median: 1.0, Std: 0.95


(np.float64(1.57), 6, 0, np.float64(1.0), np.float64(0.9513674369033238))